## Import Dependencies and Open Instrument Handles

In [1]:
import sys
import os
import time
import pyvisa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from binascii import unhexlify
import serial

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\Drivers")

from Driver_TECSource_5240 import *
from Driver_PM_PM5020 import *
from Driver_OSA_MS9740B import *
from Driver_LaserSource_71440108 import *

In [2]:
rm = pyvisa.ResourceManager()

tec = TECSource_5240()
tec.open(rm,'ASRL5::INSTR')

pm = PM_PM5020()
pm.open(rm)

osa = OSA_MS9740B()
osa.open(rm)

ld = LaserSource_71440108()
ld.open(rm)

OSA Attenuation Status:  ON



## Initialization

In [3]:
ld.set_output_off(1)
ld.set_output_off(2)
ld.set_output_off(3)
ld.set_output_on(4)

In [4]:
T_Start = 25
T_Stop = 50
T_Step = 5
t_wait = 45
n = int((T_Stop-T_Start)/T_Step+1)
pm.set_units(1,"W")

wavelength_start = 1565
wavelength_stop = 1585
wavelength_points = 2001
OSA_Resolution = 0.03
OSA_VBW = 1000
timeout = 10000

bias_start = 0
bias_stop = 500
bias_step = 1
bias_current = np.linspace(bias_start,bias_stop,int((bias_stop-bias_start)/bias_step) + 1)

Light_engine = "22-49-005"
Channel = 4

osa.set_timeout(20000)

channel1 = 1
channel2 = 2

## Measurement Routine (Temperature Ramp)

In [5]:
meas_df = pd.DataFrame()
t0 = time.time()

In [69]:
for k in range(0,n,1):
    T_Set = T_Start+k*T_Step
    tec.set_temperature(T_Set)
    time.sleep(t_wait)
    for j in range(0,int(((bias_stop-bias_start)/bias_step)+1),bias_step):
        
        ld.set_laser_current(Channel,bias_current[j])
        
        T_read = tec.get_temperature()
        t_meas = time.time()-t0
        
        Voltage_V = ld.get_laser_voltage(Channel)        
        P_out = pm.get_power(1)
        meas_df_temp = pd.DataFrame({'Time':t_meas, 'Light_engine':Light_engine, 'Bias_current_mA':bias_current[j], 'Voltage_V':Voltage_V, 'Temperature_Set_C':T_Set, 'Temperature_C':T_read, 'Channel':channel1, 'Power_dBm':P_out},index=[0])
        data = [meas_df_temp, meas_df]
        meas_df = pd.concat(data,ignore_index=True,sort=False)
        
        Voltage_V = ld.get_laser_voltage(channel2)
        P_out = pm.get_power(2)
        meas_df_temp = pd.DataFrame({'Time':t_meas, 'Light_engine':Light_engine, 'Bias_current_mA':bias_current[j], 'Voltage_V':Voltage_V, 'Temperature_Set_C':T_Set, 'Temperature_C':T_read, 'Channel':channel2, 'Power_dBm':P_out},index=[0])
        data = [meas_df_temp, meas_df]
        meas_df = pd.concat(data,ignore_index=True,sort=False)
    ld.set_laser_current(channel1,0)
    ld.set_laser_current(channel2,0)

## View, Plot and Save Data

In [70]:
meas_df

,Time,Light_engine,Bias_current_mA,Voltage_V,Temperature_Set_C,Temperature_C,Channel,Power_dBm
0,1153.950177,22-49-001,500.0,3.163,50,50.07,2,0.052148
1,1153.950177,22-49-001,500.0,2.938,50,50.07,1,0.056945
2,1153.870087,22-49-001,499.0,3.157,50,50.07,2,0.052165
3,1153.870087,22-49-001,499.0,2.938,50,50.07,1,0.056925
4,1153.806185,22-49-001,498.0,3.152,50,50.07,2,0.052240
...,...,...,...,...,...,...,...,...
12019,45.392776,22-49-001,2.0,1.380,25,25.00,3,0.000010
12020,45.312694,22-49-001,1.0,1.760,25,25.00,4,0.017962
12021,45.312694,22-49-001,1.0,1.380,25,25.00,3,0.013063
12022,45.248718,22-49-001,0.0,2.554,25,25.00,4,0.048818


In [74]:
meas_df.to_csv(r"22-49-005_C44_Channel_4.csv")

## Close Instrument Handles

In [8]:
pm.close()
tec.close()
osa.close()
ld.close()

In [75]:
tec.set_temperature(23)

In [72]:
ld.set_laser_current(1,0)
ld.set_laser_current(2,0)